# PROVIDE A RANDOM DELAY TO ACTIVITIES

In [48]:
# Import dependencies
import datetime
import dateutil
import openclsim.model as model
import openclsim.plugins as plugins
import pandas as pd
import scipy.stats as st
import simpy

In [63]:
# Create a simulation environment
class SimulationEnvironment(object):
    """ A OpenCLSim simulation object.

    Use the class methods to define the activities and start the
    simulation. In this example, a basic activity is modelled using 
    the `model.BasicActivity` class and the length of the activity is
    extended using a random delay with the `model.HasDelayPlugin`. For
    further reference, check the `DelayBasicActivity` class below.

    Parameters
    ----------
        start_date: datetime.datetime
            A datetime object specifying the simulation start date.

    """


    def __init__(self, start_date: datetime.datetime):

        # Define start date and time of simulation
        self.start_date = start_date
        start_utc = start_date.replace(tzinfo=dateutil.tz.UTC)
        start_epoch = start_utc.timestamp()

        # Initialise a SimPy simulation environment
        self.env = simpy.Environment(initial_time=start_epoch)

    def define_activities(self):
        """ Use this function to define your activities. """

        # In this example we define a basic activity
        delay_by = st.uniform(loc=0, scale=1)  # The delay as RANDOM variable.
        activity = DelayedBasicActivity(
            env=self.env,  # The SimPy environment
            duration=3600,  # The activity takes an hour
            registry={} ,  # 
            name='delayed basic activity',  # Description
            delay_percentage=delay_by,  # The delay in percentages
            postpone_start=True  # For sequential or while activities 
        )

        # We simply repeat the activity for demonstration purposes
        sequence = model.SequentialActivity(
            env=self.env,
            name='sequential activity',
            sub_processes=[activity, activity, activity],
            registry={}
        )

        return [activity]

    def execute_simulation(self):
        """ Function starts the simulation. """

        self.activities = self.define_activities()
        self.env.run()
        return 'SUCCESSFUL'

    @property
    def event_log(self):
        """ Function returns the event log """
        pass


# Define the delayed basic activity using inheritance.
class DelayedBasicActivity(plugins.HasDelayPlugin, model.BasicActivity):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

In [66]:
sim = SimulationEnvironment(datetime.datetime(2021, 1, 1))
res = sim.execute_simulation()
pd.DataFrame(sim.activities[0].log)

,Timestamp,ActivityID,ActivityState,ObjectState,ActivityLabel
0,2021-01-01 00:00:00.000000,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,START,{},{}
1,2021-01-01 01:00:00.000000,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,STOP,{},{}
2,2021-01-01 01:00:00.000000,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,WAIT_START,{},"{'type': 'plugin', 'ref': 'delay'}"
3,2021-01-01 01:00:21.362473,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,WAIT_STOP,{},"{'type': 'plugin', 'ref': 'delay'}"
4,2021-01-01 01:00:21.362473,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,START,{},{}
5,2021-01-01 02:00:21.362473,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,STOP,{},{}
6,2021-01-01 02:00:21.362473,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,WAIT_START,{},"{'type': 'plugin', 'ref': 'delay'}"
7,2021-01-01 02:00:32.394759,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,WAIT_STOP,{},"{'type': 'plugin', 'ref': 'delay'}"
8,2021-01-01 02:00:32.394759,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,START,{},{}
9,2021-01-01 03:00:32.394759,62fe0daf-a456-4aa3-bcbc-9d0b6a2a5c99,STOP,{},{}
